In [71]:
import pandas as pd
import os
from sqlalchemy import create_engine
from urllib.parse import quote_plus
from datetime import datetime, timedelta

## Connections + Path

In [72]:
server = 'QTHONG'
database = 'SC_Pipeline'
connection_string = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};Trusted_Connection=yes;'
params = quote_plus(connection_string)
sqlalchemy_connection_string = f"mssql+pyodbc:///?odbc_connect={params}"
engine = create_engine(sqlalchemy_connection_string)

In [73]:
pwd = os.getcwd()
csv_directory = os.path.join(pwd, "data")  # The path to  CSV files
print(os.listdir(csv_directory))

['consignment_by_customer.csv', 'dim_customer.csv', 'dim_material.csv', 'log_shipment_rec.csv', 'po_by _vendor.csv', 'so_by_customer.csv', 'stock_by_material.csv', 'sto_rec.csv', 'weekly_open_order.csv']


## Handle Excel Date Format

In [74]:
def excel_serial_date_to_datetime(serial_date):
    try:
        serial_date = float(serial_date)  # Ensure the value is a float
        base_date = datetime(1899, 12, 30)
        delta = timedelta(days=serial_date)
        return base_date + delta
    except ValueError:
        # Return None or some default value if conversion fails
        return None

## Handle String Date + Excel Date Format

In [75]:
def convert_date_column(date_value):
    if pd.isnull(date_value):
        return None
    
    try:
        # Attempt to convert the value to float, indicating it could be an Excel serial date
        numeric_value = float(date_value)
        return excel_serial_date_to_datetime(numeric_value)
    except ValueError:
        # If conversion fails, attempt to parse as a string-formatted date
        return pd.to_datetime(date_value, errors='coerce', dayfirst=True)  # Assuming day-first format


## Create Tables

In [76]:
def read_csv_and_create_table(file_path, table_name, date_columns=None):
    try:
        df = pd.read_csv(file_path)
        
        # Clean the column names
        df.columns = df.columns.str.strip().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')

        # Iterate over known date columns and convert them
        if date_columns:
            for col in date_columns:
                if col in df.columns:
                    df[col] = df[col].apply(convert_date_column)
        
        # Insert data into the database
        df.to_sql(table_name, con=engine, if_exists='replace', index=False, method=None)
        print(f'Imported {file_path} into {table_name}')
    except Exception as e:
        print(f"An error occurred with {file_path}: {e}")

## Main Loop

In [77]:
# known_date_columns = ['Document_Date', 'Delivery_Date', 'Date','Date_Entered','Revised_Due_Date','ESD']
for csv_file in os.listdir(csv_directory):
    if csv_file.endswith('.csv'):
        file_path = os.path.join(csv_directory, csv_file)
        table_name = os.path.splitext(csv_file)[0].replace(' ', '_').replace('-', '_')
        
        # If you can specify date columns directly
        read_csv_and_create_table(file_path, table_name, date_columns=known_date_columns)

Imported c:\Users\phamt\Documents\3000_a_month\Data Analyst\Personal projects\Portfoilo projects\Full Data Pipelines\data\consignment_by_customer.csv into consignment_by_customer
Imported c:\Users\phamt\Documents\3000_a_month\Data Analyst\Personal projects\Portfoilo projects\Full Data Pipelines\data\dim_customer.csv into dim_customer
Imported c:\Users\phamt\Documents\3000_a_month\Data Analyst\Personal projects\Portfoilo projects\Full Data Pipelines\data\dim_material.csv into dim_material
Imported c:\Users\phamt\Documents\3000_a_month\Data Analyst\Personal projects\Portfoilo projects\Full Data Pipelines\data\log_shipment_rec.csv into log_shipment_rec


C:\Users\phamt\AppData\Local\Temp\ipykernel_18012\87096264.py:11: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(date_value, errors='coerce', dayfirst=True)  # Assuming day-first format


Imported c:\Users\phamt\Documents\3000_a_month\Data Analyst\Personal projects\Portfoilo projects\Full Data Pipelines\data\po_by _vendor.csv into po_by__vendor


C:\Users\phamt\AppData\Local\Temp\ipykernel_18012\87096264.py:11: UserWarning: Parsing dates in %m/%d/%Y format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  return pd.to_datetime(date_value, errors='coerce', dayfirst=True)  # Assuming day-first format


Imported c:\Users\phamt\Documents\3000_a_month\Data Analyst\Personal projects\Portfoilo projects\Full Data Pipelines\data\so_by_customer.csv into so_by_customer
Imported c:\Users\phamt\Documents\3000_a_month\Data Analyst\Personal projects\Portfoilo projects\Full Data Pipelines\data\stock_by_material.csv into stock_by_material
Imported c:\Users\phamt\Documents\3000_a_month\Data Analyst\Personal projects\Portfoilo projects\Full Data Pipelines\data\sto_rec.csv into sto_rec
Imported c:\Users\phamt\Documents\3000_a_month\Data Analyst\Personal projects\Portfoilo projects\Full Data Pipelines\data\weekly_open_order.csv into weekly_open_order
